In [1]:
import operator
import random
import requests
import time
import csv
import os
import pandas as pd
import seaborn as sns
import numpy as np
import networkx as nx
from scipy import stats
from ast import literal_eval
from matplotlib import pyplot as plt
from statsmodels.stats import diagnostic
from datetime import datetime
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#colors used when comparing genders
PURPLE = 'purple'
ORANGE = '#ffb82c'

#colors used when generic comparisons
BLUE = '#0173b2'
RED = '#d55e00'

#print(sns.color_palette("colorblind").as_hex())
#sns.color_palette("colorblind")

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# 1. Load the datasets
<a class="anchor" id="load"></a>

In [3]:
DATA_PATH = './data'
MOVIE_PATH = os.path.join(DATA_PATH, 'american_movies_cleaned.tsv')
CHAR_PATH = os.path.join(DATA_PATH, 'characters_cleaned.tsv')
THE_NUMBERS_PATH = os.path.join(DATA_PATH, 'the_numbers_lifetime_cumulative.tsv')
THE_NUMBERS_PATH_YEARLY = os.path.join(DATA_PATH, 'the_numbers_yearly_score.tsv')

## 1.1 Characters dataset

In [4]:
df_chars=pd.read_csv(CHAR_PATH, sep="\t")
df_chars['m_release_date']=pd.to_datetime(df_chars['m_release_date'], errors = 'coerce')
df_chars['actor_dob']=pd.to_datetime(df_chars['actor_dob'], errors = 'coerce')
df_chars.head(2)

,wikipedia_m_ID,freebase_m_ID,m_release_date,character_name,actor_dob,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age,freebase_character_map,freebase_character_id,freebase_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,NaN,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


## 1.2 Movie dataset

In [5]:
df_movies=pd.read_csv(MOVIE_PATH, sep='\t')
df_movies['m_release_date']=pd.to_datetime(df_movies['m_release_date'], errors = 'coerce')
df_movies.head(2)

,wikipedia_m_ID,freebase_m_ID,m_name,m_release_date,m_BO_revenue,m_runtime,m_languages,m_countries,m_genres,m_genre_PCA
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,['English Language'],United States of America,"['Thriller', 'Science Fiction', 'Horror', 'Adv...","Thriller, Crime Thriller, Action, Mystery"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,['English Language'],United States of America,"['Mystery', 'Biographical film', 'Drama', 'Cri...","Mystery, Horror, Children's/Family, Drama"


# 2. Dataset preparation

## 2.1 Characters dataset

In [6]:
# Keeps just one row if an actor played several roles in a movie
df_chars.drop_duplicates(['wikipedia_m_ID', 'actor_name'], keep='first', inplace=True)

In [7]:
# Keep correct actors (you're born before releasing a movie)
df_chars = df_chars.loc[(df_chars['actor_dob'] < df_chars['m_release_date'])]

In [8]:
df_test = df_chars.copy()
df_test.drop_duplicates(subset=['actor_name', 'actor_dob'], keep='first', inplace=True)
df_test = df_test[df_test.duplicated('actor_name', keep=False)]
print("There are {} rows with same actor name but different date of birth.".format(df_test.shape[0]))
# We decided to keep those rows, 
# but add modifications on the ones whose actor name appears in the numbers (to merge the score to the true actor)

There are 515 rows with same actor name but different date of birth.


In [9]:
filter_conditions=(
    ((df_chars['actor_name'] =='Edward Burns') & (df_chars['actor_dob'] =='1879-09-21')) |
    ((df_chars['actor_name'] =='Harrison Ford') & (df_chars['actor_dob'] =='1884-03-16')) |
    ((df_chars['actor_name'] =='Kevin James') & (df_chars['actor_dob'] =='1954-11-17')) # porn actor
)
df_chars = df_chars.loc[~filter_conditions]

In [10]:
filter_conditions=(
    (df_chars['actor_name'] =='Patrick Wilson') & (df_chars['actor_dob'] =='1969-02-01') 
)
df_chars.loc[filter_conditions, 'actor_dob']= '1973-07-03'

In [11]:
filter_conditions=(
    (df_chars['actor_name'] =='Justin Long') & (df_chars['actor_dob'] =='1970-06-29') 
)
df_chars.loc[filter_conditions, 'actor_name']= 'Justin Long 2'

In [12]:
filter_conditions=(
    (df_chars['actor_name'] =='Kenny Rogers') & (df_chars['actor_dob'] =='1964-11-10') 
)
df_chars.loc[filter_conditions, 'actor_dob']= '1938-08-21'

In [13]:
filter_conditions=(
    (df_chars['actor_name'] =='Kevin Smith') & (df_chars['actor_dob'] =='1963-03-16') 
)
df_chars.loc[filter_conditions, 'actor_name']= 'Kevin Smith_NZ'

In [14]:
# Keeps only movies where we have at least two actors (as the interest is coacting)
df_chars = df_chars[df_chars.duplicated('wikipedia_m_ID', keep=False)]

In [15]:
# Keeps only characters from american movies
df_chars = df_chars.merge(df_movies[['wikipedia_m_ID', 'm_name', 'm_genre_PCA']], how='inner', on='wikipedia_m_ID')

In [16]:
# Drop columns that won't be used at all for the analysis
df_chars.drop(['freebase_character_map', 'freebase_character_id', 'freebase_actor_id', 'freebase_m_ID'], axis=1, inplace=True)

## 2.2 Movie dataset

In [17]:
# Keeps only movies where actors data is available
df_movies = df_movies.merge(df_chars.drop_duplicates('wikipedia_m_ID', keep='first'), how='inner', on=['wikipedia_m_ID', 'm_release_date', 'm_genre_PCA'])

In [18]:
# Drop columns that won't be used at all for the analysis
df_movies.drop('freebase_m_ID', axis=1, inplace=True)

# 3. Output as clean CSV

In [19]:
OUTPUT_FILE = 'characters_filtered.tsv'
OUTPUT_PATH = os.path.join(DATA_PATH, OUTPUT_FILE)
df_chars.to_csv(OUTPUT_PATH, sep="\t", index=False)

In [20]:
OUTPUT_FILE = 'american_movies_filtered.tsv'
OUTPUT_PATH = os.path.join(DATA_PATH, OUTPUT_FILE)
df_movies.to_csv(OUTPUT_PATH, sep="\t", index=False)